In [5]:
import pandas as pd
import numpy as np
from datetime import datetime



In [6]:
# 加载个体层面的数据
individual_ddf = pd.read_stata("E:/680_temp3.18DID/data/totransfer.dta")

# 加载 CPI 数据
cpi_ddf = pd.read_stata("E:/680_temp3.18DID/data/cpiursai1977-2016.dta")



In [ ]:
#
cpi_data = pd.read_stata("E:/680_temp3.18DID/data/cpiursai1977-2016.dta")

# 处理 CPI 数据，假设 CSV 格式的列名和 Stata do 文件保持一致
cpi_data = cpi_data.drop(['v14', 'avg'], axis=1)
cpi_data = cpi_data.melt(id_vars=["year"], var_name="month", value_name="cpi")
cpi_data = cpi_data[cpi_data['year'] >= 1979]

# 设置基年为2016的CPI
base_year_cpi = cpi_data[cpi_data['year'] == 2016]['cpi'].mean()
cpi_data['cpi'] = 100 * cpi_data['cpi'] / base_year_cpi

# 生成 monthdate
cpi_data['monthdate'] = pd.to_datetime(cpi_data.assign(day=1).loc[:, ['year', 'month', 'day']])


In [ ]:
# 首先，确保个体数据中有 monthdate 字段
# 这需要基于你的数据结构来生成 monthdate 字段
# 例如，如果你有年和月的列：
individual_data['monthdate'] = pd.to_datetime(individual_data.assign(day=1)[['year', 'month', 'day']])

# 合并 CPI 数据
individual_data = individual_data.merge(cpi_data[['monthdate', 'cpi']], on='monthdate', how='left')

# 调整工资以反映通货膨胀影响
individual_data['adjusted_wage'] = individual_data['earnhr'] / (individual_data['cpi'] / 100)


In [4]:
# 排除自雇工作者
individual_data = individual_data[(individual_data['class'] != 5) & (individual_data['class'] != 6) | (individual_data['year'] > 1993)]

# 处理工资数据
# 例如，创建 wagebins
individual_data['wagebins'] = np.select(
    [
        individual_data['adjusted_wage'] < 125, 
        (individual_data['adjusted_wage'] >= 125) & (individual_data['adjusted_wage'] < 3000), 
        individual_data['adjusted_wage'] >= 3000
    ], 
    [
        100, 
        np.floor(individual_data['adjusted_wage'] / 25) * 25, 
        3000
    ], 
    default=0
)



NameError: name 'individual_data' is not defined

In [ ]:
data = pd.read_stata("E:/680_temp3.18DID/data/cpiursai1977-2016.dtastate_panels_cents_QJE.dta")
data = data[data['year'].notnull()]
numberqrt = data['quarterdate'].max() - data['quarterdate'].min() + 1
minqrt = data['quarterdate'].min()
data['tagger'] = data.groupby(['wagebins', 'statenum']).cumcount() + 1
data = data[data['tagger'] == 1]
expanded_data = pd.DataFrame([
    [wb, sn, qd] for wb, sn in zip(data['wagebins'], data['statenum'])
    for qd in range(minqrt, minqrt + numberqrt)
], columns=['wagebins', 'statenum', 'quarterdate'])
# 假设 'wagebinstateshell' 已经是你通过上一步生成的 DataFrame
# 这里使用 'expanded_data' 代替 'wagebinstateshell'

merged_data = pd.merge(data, expanded_data, on=['wagebins', 'statenum', 'quarterdate'], how='outer')
for x in ['year', 'cpi', 'blackcountall', 'hispaniccountall', 'teencountall', 'hslcountall', 'hsl40countall', 'hsdcountall', 'hsd40countall', 'gendercountall', 'totalpopulation', 'whitecountall', 'countall']:
    merged_data[x] = merged_data.groupby(['statenum', 'quarterdate'])[x].transform(lambda grp: grp.fillna(grp.mean()))
for x in ['blackcount', 'dmarriedcount', 'gendercount', 'hispaniccount', 'hslcount', 'hsl40count', 'hsdcount', 'hsd40count', 'teencount', 'totpopcount', 'whitecount', 'FTE_orig']:
    merged_data[x] = merged_data[x].fillna(0)
